In [1]:
#!pip install -r requirements.txt

In [1]:
# Run evaluation
#!python detectcamera.py --weights weights/best.pt --conf .45 --source 0

In [3]:
import tkinter as tk
from tkinter import filedialog
import PIL
import cv2
import os

from PIL import ImageTk, Image

def choose_view():
    open_history=filedialog.askopenfilename(initialdir="C:/Users/karth/OneDrive/Desktop/MINIPROJECT/saveddetection", title="Select file", filetypes=(("image/video files", ("*.jpg", "*.png","*.mp4",)), ("all files", "*.*")))
    file_extension = os.path.splitext(open_history)[1].lower()
    
    # Check if the file extension matches the allowed extensions
    if file_extension == '.mp4':
    # Create a canvas to display the video
        video_window = tk.Toplevel()
        video_window.title("Video Player")
        canvas = tk.Canvas(video_window, width=640, height=480)
        canvas.pack()
        video = cv2.VideoCapture(open_history)
        while True:
            ret, frame = video.read()
            if not ret:
                break
            # Convert the frame from BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Resize the frame to fit the canvas
            frame_resized = cv2.resize(frame_rgb, (640, 480))

            # Convert the frame to an ImageTk format
            img = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(frame_resized))

            # Update the canvas with the new frame
            canvas.create_image(0, 0, anchor=tk.NW, image=img)
            canvas.update()

            # Delay between each frame (e.g., 30 milliseconds)
            video_window.after(20)

        # Release the video file and destroy the window when done
        video.release()
        video_window.destroy()
        
        

            
    else:
        # Open the selected image file
        image = Image.open(open_history)

        # Create a new Toplevel window
        image_window = tk.Toplevel()
        image_window.title("Image Viewer")

        # Create a label to display the image
        label = tk.Label(image_window)
        label.pack()

        # Convert the image to PhotoImage format
        photo = ImageTk.PhotoImage(image)

        # Set the image in the label
        label.config(image=photo)
        label.image = photo  # Store a reference to prevent garbage collection


 

    
def choose_file():
    print("hello")
    label1= tk.Label(root, text = "Video is being processes. Please Wait ! ",font=("Arial", 15))
    label1.place(relx=0.5, rely=0.5, anchor=tk.CENTER)
    
    root.filename = filedialog.askopenfilename(initialdir="/", title="Select file", filetypes=(("image/video files", ("*.jpg", "*.png","*.mp4",)), ("all files", "*.*")))
    file_path = root.filename


    
    !python detectcamnvid.py --weights weights/best.pt --conf .45 --view-img --source {file_path} --no-trace --show-fps --track --show-track 
    label1.config(text="done")


    


def open_camera():
    !python detectcamnvid.py --weights weights/best.pt --conf .45 --source 0 --no-trace --show-fps --track --show-track

root = tk.Tk()
root.title("Drone Identification")

root.geometry("800x533")
bg_image = tk.PhotoImage(file='drone.png')
bg_label = tk.Label(root, image=bg_image)
bg_label.place(relwidth=1, relheight=1)

file_image = tk.PhotoImage(file='upload.png')
file_button = tk.Button(root, image=file_image, command=choose_file)
file_button.pack(side=tk.LEFT, pady=10, padx=(200, 20), anchor=tk.N)

camera_image = tk.PhotoImage(file="open.png")
camera_button = tk.Button(root, image=camera_image, command=open_camera)
camera_button.pack(side=tk.LEFT, pady=10, padx=(20, 40), anchor=tk.N)

play_image =tk.PhotoImage(file="view.png")
play_button=tk.Button(root,image=play_image,command=choose_view)
play_button.pack(side=tk.LEFT, pady=10, padx=(40, 200), anchor=tk.N)
root.mainloop()